# Lesson 1: Your first agent with Amazon Bedrock

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


In [4]:
roleArn

'arn:aws:iam::471112550072:role/c135316a3428926l9021298t1w63742362-BedrockAgentRole-VLVDKKTfDVoo'

## Start of the lesson

In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(
    service_name='bedrock-agent', 
    region_name='us-west-2')

In [5]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [6]:
create_agent_response

{'ResponseMetadata': {'RequestId': '5b4faa97-7508-4a43-ba56-9d71acb1ad31',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 13 Feb 2025 11:02:47 GMT',
   'content-type': 'application/json',
   'content-length': '624',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5b4faa97-7508-4a43-ba56-9d71acb1ad31',
   'x-amz-apigw-id': 'F62lwH4aPHcEExA=',
   'x-amzn-trace-id': 'Root=1-67add157-738b681c2a7ea63a53bd66f3'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:471112550072:agent/YPWQQU8CYB',
  'agentId': 'YPWQQU8CYB',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::471112550072:role/c135316a3428926l9021298t1w63742362-BedrockAgentRole-VLVDKKTfDVoo',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2025, 2, 13, 11, 2, 47, 752424, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a 

In [7]:
agentId = create_agent_response['agent']['agentId']

In [8]:
from helper import *

In [9]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [12]:
# prepares the agent
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': 'a13dc0e4-8ee0-43be-9a8b-02ca5b836e03',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 13 Feb 2025 11:07:44 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a13dc0e4-8ee0-43be-9a8b-02ca5b836e03',
   'x-amz-apigw-id': 'F63UKGRmvHcEFjA=',
   'x-amzn-trace-id': 'Root=1-67add280-4f70687337fae29806b4a791'},
  'RetryAttempts': 0},
 'agentId': 'YPWQQU8CYB',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 2, 13, 11, 7, 44, 927322, tzinfo=tzlocal())}

In [11]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [14]:
# creates an alias to be able to call the agent
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)
# once the prepared alias is created we can start using the agent in a new client

ConflictException: An error occurred (ConflictException) when calling the CreateAgentAlias operation: Could not perform Create operation, since the MyAgentAlias (id: MHRQYHCH0R) with the same name MyAgentAlias already exists

In [15]:
# creates new client to interact with running agent
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [16]:
import uuid
# by creating a session id we can maintain and the conversation history will be stored inside the agent insight the service

In [20]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True, # help us understand model response
)

In [21]:
event_stream = invoke_agent_response["completion"]

In [22]:
for event in event_stream:
    print(event)

{'chunk': {'bytes': b'1. Gather the mug and your proof of purchase (such as the receipt).\n2. Contact our customer service team at [phone number] or [email] to initiate the return process.\n3. Provide the customer service representative with the details of your purchase and the reason for the return (the mug being broken).\n4. Follow any additional instructions from the customer service team to complete the return and receive your refund.\n\nPlease let me know if you have any other questions!'}}


In [23]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [24]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 
Agent's thought process:
  Okay, let's go through the steps to handle this return request:

Agent's thought process:
  Apologies, let me try that again in the correct format:

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try a different approach to get the return policy
  information.

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try a different approach to get the return policy
  information.

Observation:
  Type: FINISH

Final response:
  To return the broken mug, please bring the mug and your original
  purchase receipt to our store within 30 days of the purchase date.
  We will be happy to provide a full refund for the mug. Please let me
  know if you have any other questions!


Session ID: f903d52c-bb8f-4390-ae00-3000cbd231d9
